In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import sklearn

import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

#Ignora warning
import warnings
warnings.filterwarnings('ignore')

from scipy.io.wavfile import write
from audiomentations import TimeMask, PitchShift, TanhDistortion, AddBackgroundNoise, SpecFrequencyMask
import librosa
import os
import librosa
import numpy as np
import pandas as pd
from scipy.io.wavfile import write
from tqdm import tqdm

# Set the backend to 'sox_io' or 'soundfile'
torchaudio.set_audio_backend("sox_io")


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

- OPCIONAL

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.0 MB/s eta 0:00:00


In [ ]:
import jiwer

# Dados

Aqui vamos importar dois datasets processados manualmente.

1. Os dados de áudio: Este aquivo zip contém os áudios brutos que seram acessados pelo treinamento. Ele é subdividado por pessoa e por classe.

2. Caminhos e Transcrições: Este arquivo .csv contém os "paths" de todos os áudios juntamente com a transcrição e classe que eles pertencem.

In [ ]:
!gdown 1IVfFb7klGmfEckV7YkXF8Rqpd46aJGPI # dados de audio
!gdown 1W6xAJFeNTArzH0iTGaoHDs87RBmLk1JL # Caminhos e Transcrições

Downloading...
From (original): https://drive.google.com/uc?id=1IVfFb7klGmfEckV7YkXF8Rqpd46aJGPI
From (redirected): https://drive.google.com/uc?id=1IVfFb7klGmfEckV7YkXF8Rqpd46aJGPI&confirm=t&uuid=385f59d1-ede1-45fb-b33e-c47bfc188980
To: /content/Dados_wav.zip
100% 158M/158M [00:01<00:00, 143MB/s]
Downloading...
From: https://drive.google.com/uc?id=1W6xAJFeNTArzH0iTGaoHDs87RBmLk1JL
To: /content/audio_data.csv
100% 166k/166k [00:00<00:00, 118MB/s]


In [ ]:
%%capture
!unzip /content/Dados_wav.zip

In [ ]:
df = pd.read_csv("/content/audio_data.csv")

## Limpeza

- Consertar o caminho

Como processei esses dados no meu PC local, o Path nele tá específico pro meu PC. Aqui vamos remover esse problema.

In [ ]:
# Modify each path in df. Remove the first 55 characters and add /content/Dados_wav/ at the beggining
df['path'] = df['path'].apply(lambda x: '/content/'+ x[55:])

- Tirar transcrições falhas

Algumas transcrições falharam. Aqui estou tirando elas.

In [ ]:
cleaned_df = df[df['transcription'] != 'Transcription failed']
cleaned_df.shape

(1190, 3)

In [ ]:
# Desencargo de consciência

path = df['path'][800]

waveform, sample_rate = torchaudio.load(path)
print(sample_rate)


48000


- Encontrar maior áudio

In [ ]:
import torchaudio
from tqdm import tqdm

def find_longest_audio_length(df):
    max_length = 0
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        audio_path = row['path']
        waveform, sample_rate = torchaudio.load(audio_path)
        length = waveform.shape[1]
        if length > max_length:
            max_length = length
    return max_length

# Find the longest audio length in the dataset
LONGEST_LENGTH = find_longest_audio_length(cleaned_df)
print(f"The longest audio length is: {LONGEST_LENGTH}")


100%|██████████| 1190/1190 [00:13<00:00, 87.10it/s] 

The longest audio length is: 144648


In [ ]:
LONGEST_LENGTH = 144648

## Aumento de dados

In [ ]:
!gdown 1FERZhKvNKcdpFGdEhSL7Uq3EcIU8k7Qd

Downloading...
From: https://drive.google.com/uc?id=1FERZhKvNKcdpFGdEhSL7Uq3EcIU8k7Qd
To: /content/bg_noises.zip
100% 3.45M/3.45M [00:00<00:00, 179MB/s]


In [ ]:
!unzip /content/bg_noises.zip

Archive:  /content/bg_noises.zip
  inflating: bg_noises/ruido_1.wav   
  inflating: bg_noises/ruido_2.wav   


In [ ]:
%%capture
!pip install librosa==0.9.0 audiomentations datasets torchinfo

In [ ]:
def data_aug(dataset, aug_function):
    new_file_names = []
    new_labels = []
    transcriptions = []

    for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
        file_path = os.path.join(row['path'])
        audio, sr = librosa.load(file_path, sr=None)
        noise_folder = os.path.dirname(file_path)

        augmented_audio, name = aug_function(audio, sample_rate=sr)

        new_file_name = f"{os.path.dirname(file_path)}/{name}_{index}_aug.wav"

        if not os.path.exists(new_file_name):
            new_file_names.append(new_file_name)
            new_labels.append(row['class'])
            transcriptions.append(row['transcription'])

            write(os.path.join(new_file_name), sr, (augmented_audio * 32767).astype(np.int16))

    aug_data = pd.DataFrame()
    aug_data['path'] = new_file_names
    aug_data['class'] = new_labels
    aug_data['transcription'] = transcriptions

    return aug_data


In [ ]:
def augment_audio_with_background_noise(audio, sample_rate, noise_folder = '/content/bg_noises'):
    name = 'background_noise'
    background_noise_files = os.listdir(noise_folder)
    noise_file = os.path.join(noise_folder, np.random.choice(background_noise_files))
    noise_audio, _ = librosa.load(noise_file, sr=sample_rate)
    background_noise_aug = AddBackgroundNoise(sounds_path=noise_folder, p=1.0)
    augmented_audio = background_noise_aug(audio, sample_rate=sample_rate)
    return augmented_audio, name

def augment_audio_with_tanh_distortion(audio, sample_rate):
    name = 'tanh_distortion'
    tanh_dist_aug = TanhDistortion(p=1.0)
    augmented_audio = tanh_dist_aug(audio, sample_rate=sample_rate)
    return augmented_audio, name

def augment_audio_with_spec_frequency_mask(audio, sample_rate):
    name = 'spec_frequency_mask'
    spec_aug = SpecFrequencyMask()
    augmented_audio = spec_aug(audio)
    return augmented_audio, name

def augment_audio_with_pitch_shift(audio, sample_rate):
    name = 'pitch_shift'
    pitch_shift_aug = PitchShift(p=1.0)
    augmented_audio = pitch_shift_aug(audio, sample_rate=sample_rate)
    return augmented_audio, name

def augment_audio_with_time_mask(audio, sample_rate):
    name = 'time_mask'
    time_mask_aug = TimeMask(min_band_part=0.1, max_band_part=0.15, fade=True, p=1.0)
    augmented_audio = time_mask_aug(audio, sample_rate=sample_rate)
    return augmented_audio, name

In [ ]:
aug1_data = data_aug(cleaned_df, augment_audio_with_background_noise)
aug2_data = data_aug(cleaned_df, augment_audio_with_tanh_distortion)
# aug3_data = data_aug(cleaned_df, augment_audio_with_spec_frequency_mask) -> Esse tá dando problema
aug4_data = data_aug(cleaned_df, augment_audio_with_pitch_shift)
aug5_data = data_aug(cleaned_df, augment_audio_with_time_mask)

100%|██████████| 1190/1190 [00:00<00:00, 1234.68it/s]


In [ ]:
cleaned_aug_df = pd.concat([cleaned_df, aug1_data])
cleaned_aug_df = pd.concat([cleaned_df, aug2_data])
# cleaned_aug_df = pd.concat([cleaned_aug_df, aug3_data])
cleaned_aug_df = pd.concat([cleaned_aug_df, aug4_data])
cleaned_aug_df = pd.concat([cleaned_aug_df, aug5_data])

print("Número total de amostras após o aumento de dados:", len(cleaned_aug_df))

Número total de amostras após o aumento de dados: 1190


In [ ]:
cleaned_aug_df.isna().sum()

path             0
transcription    0
class            0
dtype: int64

## Separação de treino e teste

In [ ]:
# prompt: Split cleaned_aug_df into train and test. Make sure there are no augmentated data on the test dataset. Every augmented data has "aug.wav" ending. Include a portion of the not augmentated data on train as well

train_df = cleaned_aug_df[cleaned_aug_df['path'].str.contains("aug.wav")].sample(frac=0.8, random_state=24)
test_df = cleaned_aug_df[~cleaned_aug_df['path'].str.contains("aug.wav")]
train_df = pd.concat([train_df, cleaned_aug_df[~cleaned_aug_df['path'].str.contains("aug.wav")].sample(frac=0.2, random_state=24)])


# Models

- Importar modelos

In [ ]:
# Initialize the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese").to("cuda")


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53-portuguese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53-portuguese and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

## Functions

Esta função são responsáveis por processar os áudios e extrair as features necessárias.

In [ ]:
def pad_waveform(waveform, target_length):
    current_length = waveform.shape[1]
    if current_length < target_length:
        padding = target_length - current_length
        waveform = torch.nn.functional.pad(waveform, (0, padding), "constant", 0)
    return waveform

def extract_features(audio_file, target_length=LONGEST_LENGTH):  # Example target length: 10 seconds
    # Load audio file
    waveform, sample_rate = torchaudio.load(audio_file)

    # Pad waveform to ensure consistent length
    waveform = pad_waveform(waveform, target_length)

    # Process the waveform to match the model's expected input
    input_values = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt", padding=True).input_values

    return input_values

def collate_fn(batch):
    input_values = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    transcriptions = [item[2] for item in batch]

    # Pad input_values and labels
    input_values = torch.nn.utils.rnn.pad_sequence(input_values, batch_first=True, padding_value=processor.tokenizer.pad_token_id)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id)

    return input_values, labels, transcriptions



## Classes

A seguinte classe implementa o dataset no formato desejado para o modelo.

In [ ]:
import torch
from torch.utils.data import Dataset
import torchaudio
import torchaudio.transforms as T

# Assuming `processor` is already defined
# processor = ...

class ASRDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.resampler = T.Resample(orig_freq=48000, new_freq=16000)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        audio_path = self.dataset.iloc[idx]['path']
        transcription = self.dataset.iloc[idx]['transcription']
        waveform, sample_rate = torchaudio.load(audio_path)

        if sample_rate != 16000:
            waveform = self.resampler(waveform)

        input_values = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors='pt').input_values
        labels = processor.tokenizer(transcription, return_tensors='pt').input_ids
        return input_values.squeeze(), labels.squeeze(), transcription



## Train script

In [ ]:
# Create train and test dataset objects
train_dataset = ASRDataset(train_df)
test_dataset = ASRDataset(test_df)

In [ ]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# Create DataLoader for train and test sets with an increased batch size
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

num_epochs = 15
learning_rate = 1e-5  # Reduced learning rate
max_norm = 1.0  # Maximum norm for gradient clipping
weight_decay = 1e-2  # Weight decay for regularization

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)  # Learning rate scheduler

criterion = torch.nn.CTCLoss(blank=processor.tokenizer.pad_token_id, zero_infinity=True)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch in tqdm(train_loader):
        input_values, labels, transcription = batch

        # Move tensors to the appropriate device
        input_values = input_values.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_values, labels=labels)

        # Ensure outputs contain the loss attribute
        if hasattr(outputs, 'loss'):
            loss = outputs.loss
        else:
            raise ValueError("Model outputs do not contain 'loss' attribute")

        # Backward pass and optimization
        loss.backward()

        # Clip the gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()

        epoch_loss += loss.item()

    scheduler.step()  # Update the learning rate

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}")

# Save the model
model.save_pretrained("asr_model")
processor.save_pretrained("asr_processor")

100%|██████████| 15/15 [00:26<00:00,  1.78s/it]


Epoch 1/15, Loss: 248.7935385386149


100%|██████████| 15/15 [00:24<00:00,  1.61s/it]


Epoch 2/15, Loss: 112.55631980895996


100%|██████████| 15/15 [00:24<00:00,  1.66s/it]


...

100%|██████████| 15/15 [00:23<00:00,  1.57s/it]


Epoch 14/15, Loss: 92.18804321289062


100%|██████████| 15/15 [00:24<00:00,  1.65s/it]


Epoch 15/15, Loss: 91.08114840189616


[]

In [ ]:
# Save the model
model.save_pretrained("asr_model_augmented")
processor.save_pretrained("asr_processor_augmented")

[]

In [ ]:
# delete /content/asr_model
!rm -rf /content/asr_processor_1_augmented

In [ ]:
# move asr models to /content/drive/MyDrive/BIA/7º Periodo/PAV/Trabalho Final/Modelos/Pipeline 1
!mv /content/asr_model_augmented '/content/drive/MyDrive/BIA/7º Periodo/PAV/Trabalho Final/Modelos/Pipeline 1'
!mv /content/asr_processor_augmented '/content/drive/MyDrive/BIA/7º Periodo/PAV/Trabalho Final/Modelos/Pipeline 1'

## Evaluate model

Aqui vamos importar nosso modelo treinado e avaliar a performance do modelo

In [ ]:
# Initialize the processor and model
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/BIA/7º Periodo/PAV/Trabalho Final/Modelos/Pipeline 1/asr_processor_augmented")
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/BIA/7º Periodo/PAV/Trabalho Final/Modelos/Pipeline 1/asr_model_augmented").to("cuda")

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53-portuguese").to("cuda")


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53-portuguese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53-portuguese and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [ ]:
# Evaluation function
def evaluate(model, dataloader, processor):
    model.eval()
    total_loss = 0
    predictions, references = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_values, labels, transcripts = batch
            input_values = input_values.to("cuda")
            labels = labels.to("cuda")

            outputs = model(input_values, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            predicted_ids = torch.argmax(logits, dim=-1)
            decoded_preds = processor.batch_decode(predicted_ids, skip_special_tokens=True)
            decoded_refs = [processor.tokenizer.decode(label_ids, skip_special_tokens=True) for label_ids in labels]

            predictions.extend(decoded_preds)
            references.extend(transcripts)

    avg_loss = total_loss / len(dataloader)
    wer = jiwer.wer(references, predictions)
    return avg_loss, wer, predictions, references

# Evaluate the model on the test set
test_loss, test_wer, predictions, references = evaluate(model, test_loader, processor)


100%|██████████| 75/75 [00:49<00:00,  1.52it/s]


In [ ]:
print(f"Test Loss: {test_loss}")
print(f"Test WER: {test_wer}")

# Print some example predictions
for i in range(100):
    print(f"Reference: {references[i]}")
    print(f"Prediction: {predictions[i]}")
    print("---")

Test Loss: -149.9929384358724
Test WER: 0.5007000840100813
Reference: pode ir para cima já
Prediction: pode ir paracimajá
---
Reference: você pode subir
Prediction: você pode subir
---
Reference: só su
Prediction: só suba
---
Reference: voa para cima
Prediction: vouapracima
---
Reference: sobe
Prediction: some
---
Reference: vai para cima
Prediction: lápracima
---
Reference: suba cuidadosamente
Prediction: suba cuidadosamente
---
Reference: Levanta logo
Prediction: levanta logo
---
Reference: para cima
Prediction: paracima
---
Reference: levanta devagar
Prediction: levanta devagar
---
Reference: vamos subir
Prediction: vamos subir
---
Reference: sobe rápido
Prediction: sobe rápido
---
Reference: anda sobe
Prediction: anda sobe
---
Reference: vai para cima
Prediction: vai para cima
---
Reference: quero que
Prediction: quero que suba
---
Reference: pode subir
Prediction: pode subir
---
Reference: vai lá para cima
Prediction: vai lá para cima
---
Reference: pode subir
Prediction: pode sub

In [ ]:
# zip asr_model and processor
!zip -r /content/asr_model_1.zip /content/asr_model
!zip -r /content/asr_processor_1.zip /content/asr_processor

  adding: content/asr_model/ (stored 0%)
  adding: content/asr_model/model.safetensors (deflated 7%)
  adding: content/asr_model/config.json (deflated 64%)
  adding: content/asr_processor/ (stored 0%)
  adding: content/asr_processor/vocab.json (deflated 55%)
  adding: content/asr_processor/special_tokens_map.json (deflated 45%)
  adding: content/asr_processor/tokenizer_config.json (deflated 71%)
  adding: content/asr_processor/preprocessor_config.json (deflated 39%)
